In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import argparse
import sys
from gensim.models import Word2Vec
import re

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import seed_everything
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

sys.path.append("../")

from utils.data_processing import get_process_data
from utils.data_loader import load_document
from utils.loss import ListNet
from utils.eval import retrieval_normalized_dcg_all, retrieval_precision_all

# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def get_freer_gpu():
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free > tmp')
    memory_available = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    return int(np.argmax(memory_available))

In [3]:
device = f"cuda:{get_freer_gpu()}"
config = {
    "topk":[10,30,50]
}

In [4]:
seed = 123
seed_everything(123)

Global seed set to 123


123

In [5]:
from octis.dataset.dataset import Dataset
raw_dataset = Dataset()
raw_dataset.fetch_dataset("20NewsGroup")

In [6]:
corpus = raw_dataset.get_corpus()
raw_documents = [" ".join(i) for i in corpus]

In [7]:
docvec = np.load("../data/docvec_20NG_SBERT_768d.npy")
dim = 768

In [8]:
# get TF-IDF score
vectorizer = TfidfVectorizer(min_df=10,stop_words="english")
importance_score = vectorizer.fit_transform(raw_documents).todense()

In [9]:
vocab_size = len(vectorizer.vocabulary_)
print(f"Vocab size:{vocab_size}")

Vocab size:1588


In [10]:
class IDEDataset(Dataset):
    def __init__(self, 
                 doc_vectors,
                 weight_ans):
        
        assert len(doc_vectors) == len(weight_ans)

        self.doc_vectors = torch.FloatTensor(doc_vectors)
        self.weight_ans = torch.FloatTensor(weight_ans)        
        self.weight_ans_s = torch.argsort(self.weight_ans, dim=1, descending=True)
        self.topk = torch.sum(self.weight_ans > 0, dim=1)
        
    def __getitem__(self, idx):
        return self.doc_vectors[idx], self.weight_ans[idx], self.weight_ans_s[idx], self.topk[idx]

    def __len__(self):
        return len(self.doc_vectors)

In [11]:
dataset = IDEDataset(docvec, importance_score)
train_length = int(len(dataset)*0.6)
valid_length = int(len(dataset)*0.2)
test_length = len(dataset) - train_length - valid_length

full_loader = DataLoader(dataset, batch_size=128)
train_dataset, valid_dataset, test_dataset = random_split(
    dataset, lengths=[train_length, valid_length,test_length],
    generator=torch.Generator().manual_seed(42)
    )

In [12]:
train_loader = DataLoader(
    train_dataset, batch_size=128, 
    shuffle=True, pin_memory=True,
)
valid_loader = DataLoader(
    valid_dataset, batch_size=128, shuffle=False, pin_memory=True,drop_last=True)
test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False)

In [13]:
class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.word_embedding = nn.Parameter(torch.randn(hidden_dim, output_dim))
        self.transform = nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Sigmoid(),
        )
    def forward(self, x):
        docvec = self.transform(x)
        decoded = torch.sigmoid(torch.matmul(docvec,self.word_embedding))
        return decoded
    
    def load_pretrianed(self,word_embedding):
        self.word_embedding = nn.Parameter(torch.FloatTensor(word_embedding))

In [14]:
def evaluate_Decoder(model, data_loader):
    results = defaultdict(list)
    model.eval()
        
    # predict all data
    for data in data_loader:
        doc_embs, target, _, _ = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
                
        pred = model(doc_embs)
    
        # Precision
        precision_scores = retrieval_precision_all(pred, target, k=config["topk"])
        for k, v in precision_scores.items():
            results['precision@{}'.format(k)].append(v)
        
        # NDCG
        ndcg_scores = retrieval_normalized_dcg_all(pred, target, k=config["topk"])
        for k, v in ndcg_scores.items():
            results['ndcg@{}'.format(k)].append(v)
        
    for k in results:
        results[k] = np.mean(results[k])

    return results

In [15]:
decoder = Decoder(input_dim=dim,hidden_dim=1024,output_dim=vocab_size)
optimizer = torch.optim.Adam(decoder.parameters(), lr = 1e-4)
# initialize parameters
for p in decoder.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# decoder.load_pretrianed(pretrain_wordembedding)
decoder = decoder.to(device)

In [16]:
# early stop settings 
stop_rounds = 3
no_improvement = 0
best_score = None 


for epoch in range(100):
    # Training
    decoder.train()
    train_loss = []
    for batch in tqdm(train_loader, desc="Training"):
        batch = [i.to(device) for i in batch]
        doc_embs, target, _, _ = batch
        target = torch.nn.functional.normalize(target.to(device), dim=1)
        decoded = torch.nn.functional.normalize(decoder(doc_embs), dim=1)
        loss = ListNet(decoded, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss.append(loss.item())

    print(f"[Epoch {epoch+1:02d}]")
    res = evaluate_Decoder(decoder, valid_loader)
    for key,val in res.items():
        print(f"{key}:{val:.4f}")
        
     # early stopping 
    current_score = res["precision@10"]
    if best_score == None:
        best_score = current_score
        continue
    if current_score < best_score:
        no_improvement += 1
    if no_improvement >= stop_rounds:
        print("Early stopping...")
        break 
    if current_score > best_score:
        no_improvement = 0
        best_score = current_score

Training:   0%|          | 0/77 [00:00<?, ?it/s]

[Epoch 01]
precision@10:0.1843
precision@30:0.1287
precision@50:0.1082
ndcg@10:0.0938
ndcg@30:0.1157
ndcg@50:0.1360
ndcg@all:0.3672


Training:   0%|          | 0/77 [00:00<?, ?it/s]

[Epoch 02]
precision@10:0.1971
precision@30:0.1657
precision@50:0.1385
ndcg@10:0.0925
ndcg@30:0.1511
ndcg@50:0.1805
ndcg@all:0.3903


Training:   0%|          | 0/77 [00:00<?, ?it/s]

...

[Epoch 45]
precision@10:0.6212
precision@30:0.3490
precision@50:0.2546
ndcg@10:0.6479
ndcg@30:0.6452
ndcg@50:0.6629
ndcg@all:0.7691


Training:   0%|          | 0/77 [00:00<?, ?it/s]

[Epoch 46]
precision@10:0.6181
precision@30:0.3500
precision@50:0.2550
ndcg@10:0.6458
ndcg@30:0.6447
ndcg@50:0.6622
ndcg@all:0.7685


Training:   0%|          | 0/77 [00:00<?, ?it/s]

[Epoch 47]
precision@10:0.6195
precision@30:0.3494
precision@50:0.2551
ndcg@10:0.6494
ndcg@30:0.6472
ndcg@50:0.6648
ndcg@all:0.7703
Early stopping...


In [17]:
print("Testing...")
res = evaluate_Decoder(decoder, test_loader)
for key,val in res.items():
    print(f"{key}:{val:.4f}")

Testing...
precision@10:0.6097
precision@30:0.3402
precision@50:0.2491
ndcg@10:0.6482
ndcg@30:0.6449
ndcg@50:0.6634
ndcg@all:0.7690


## Empirical evaluation

In [18]:
import colored
from colored import stylize
from colored import fore, back, style
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [23]:
labels = raw_dataset.get_labels()
test_indices = test_dataset.indices
test_documents = [raw_documents[i] for i in test_indices]
test_labels = [labels[i] for i in test_indices]
test_embedding = torch.FloatTensor(docvec[test_indices])
test_importance = torch.FloatTensor(np.array(importance_score)[test_indices])
itos = vectorizer.get_feature_names()

### Train topic model for comparision

In [19]:
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords

train_documents = [raw_documents[i] for i in train_dataset.indices]
sp = WhiteSpacePreprocessingStopwords(train_documents, "english")
preprocessed_documents, unpreprocessed_documents, vocab = sp.preprocess()

In [20]:
from contextualized_topic_models.models.ctm import CombinedTM, ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file

qt = TopicModelDataPreparation("all-mpnet-base-v2")

training_dataset = qt.fit(text_for_contextual=unpreprocessed_documents, text_for_bow=preprocessed_documents)

ctm = CombinedTM(bow_size=len(qt.vocab), contextual_size=768, n_components=50) # 50 topics

ctm.fit(training_dataset) # run the model

Batches:   0%|          | 0/49 [00:00<?, ?it/s]

Epoch: [100/100]	 Seen Samples: [978500/978500]	Train Loss: 341.2394142022228	Time: 0:00:05.163202: : 100it [08:30,  5.10s/it]
Sampling: [20/20]: : 20it [01:26,  4.33s/it]


In [21]:
topic_words = ctm.get_topics(10)

In [24]:
sp = WhiteSpacePreprocessingStopwords(test_documents, "english")
preprocessed_documents, unpreprocessed_documents, vocab = sp.preprocess()
test_dataset = qt.transform(text_for_contextual=unpreprocessed_documents, text_for_bow=preprocessed_documents)

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [25]:
test_topics = ctm.get_doc_topic_distribution(test_dataset, n_samples=20) # returns a (n_documents, n_topics) matrix with the topic distribution of each document

Sampling: [20/20]: : 20it [01:20,  4.05s/it]


### Decoder output and analysis

In [26]:
check_id = 2 #1200
K = 10
raw_doc = test_documents[check_id]
category = test_labels[check_id]
embedding = test_embedding[check_id].unsqueeze(0).to(device)
prediction = torch.topk(decoder(embedding)[0],k=K).indices.detach().cpu().numpy()
important_words = torch.topk(test_importance[check_id],k=K).indices
print(category)

# calculate score
precision_scores = retrieval_precision_all(decoder(embedding).detach().cpu(), test_importance[check_id].unsqueeze(0), k=config["topk"])
ndcg_scores = retrieval_normalized_dcg_all(decoder(embedding).detach().cpu(), test_importance[check_id].unsqueeze(0), k=config["topk"])
print(precision_scores)
print(ndcg_scores)

# transform to word
prediction = [itos[i] for i in prediction]
important_words = [itos[i] for i in important_words]

sci.crypt
{10: 0.6000000238418579, 30: 0.36666667461395264, 50: 0.3199999928474426}
{10: 0.3357486426830292, 30: 0.3524261713027954, 50: 0.3837764859199524, 'all': 0.5948744416236877}


In [27]:
print(style.BOLD + back.LIGHT_SKY_BLUE_1 + 'Ground truth'+ style.RESET)
for word in raw_doc.split():
    if word in important_words:
        print (fore.LIGHT_BLUE + style.BOLD + word + style.RESET,end = " ")
    else:
        print(word, end=' ')

Ground truth
suggest common distribution private key public key system encrypt posting theory work fine long key remain secure practice good idea check violation net rule practice good idea check key good idea check post forward site make chain work problem discussion group travel facility control member problem mailing list approach fun 

In [28]:
print(style.BOLD + back.LIGHT_PINK_1 + 'Prediction'+ style.RESET)
for word in raw_doc.split():
    if word in prediction:
        print (fore.LIGHT_RED + style.BOLD + word + style.RESET,end = " ")
    else:
        print(word, end=' ')

Prediction
suggest common distribution private key public key system encrypt posting theory work fine long key remain secure practice good idea check violation net rule practice good idea check key good idea check post forward site make chain work problem discussion group travel facility control member problem mailing list approach fun 

In [29]:
pred_topic = np.argsort(test_topics[check_id])[-2:][::-1]
for topicID in pred_topic:
    print("Topic id: %s, Probability: %.2f" %(topicID, test_topics[check_id][topicID]))
    print(" ".join(topic_words[topicID]))

Topic id: 42, Probability: 0.14
privacy internet protect technology encryption ensure policy law security network
Topic id: 46, Probability: 0.13
key block bit chip encrypt number attack serial session message
